In [1]:
import tweepy
from twarc import Twarc2, expansions
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
from nltk.stem.porter import PorterStemmer as PS
import time
import schedule
import datetime
import gc

In [ ]:
#lemmatization
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence

In [ ]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')
doc = nlp('Her bags are better than his.')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
#lemmatization
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def lemma(txt):
    s = ''
    doc = nlp(txt)
    
    for sent in doc.sentences:
        for word in sent.words:
            s += word.lemma + ' '
    
    return s

In [ ]:
df1 = pd.read_csv('./tweet-sentiment/2021-01_slist.csv',usecols = ['Compound'])
df2 = pd.read_csv('./tweet-sentiment/2021-01_slist_Number0.csv', usecols = ['Compound'])

In [ ]:
df1["bool"] = (df1['Compound'] == df2['Compound'])

In [ ]:
df_comp = df1.compare(df2)
df_comp

In [ ]:
#del df3
gc.collect()

In [ ]:
df = pd.read_csv('./binance/BTCUSDT/1d/BTCUSDT-1d-2021-01.csv',usecols = [,'open','close'])
df.head(31)

In [ ]:
df1 = df.pct_change()
print(df1['open'])

In [ ]:
df1.head(100)

In [ ]:
print(df.diff())

In [ ]:
a=1612051200000
b=1611964800000
print((a-b)/24,(a-b)/(24*3600))

In [ ]:
import pandas as pd
print (pd.__file__)

In [ ]:
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-01.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-02.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-03.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-04.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-05.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-06.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')


In [8]:
timespan = '5m'
files = [f'tweet-sentiment/{timespan}/2021-01.csv',
             f'tweet-sentiment/{timespan}/2021-02.csv',
             f'tweet-sentiment/{timespan}/2021-03.csv',
             f'tweet-sentiment/{timespan}/2021-04.csv',
             f'tweet-sentiment/{timespan}/2021-05.csv',
             f'tweet-sentiment/{timespan}/2021-06.csv']
datas = []
df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file)
    datas.append(data)

df = pd.concat(datas).reset_index(drop=True)

print(df['trend'].value_counts())

0    17497
2    17338
1    17293
Name: trend, dtype: int64


In [5]:
del df

In [17]:
df = pd.read_csv(f'tweet-sentiment/1d/2021-01.csv')

In [38]:
n=2
timespan=str("1d")

files = [f'tweet-sentiment/{timespan}/2021-01.csv',
         f'tweet-sentiment/{timespan}/2021-02.csv',
         f'tweet-sentiment/{timespan}/2021-03.csv',
         f'tweet-sentiment/{timespan}/2021-04.csv',
         f'tweet-sentiment/{timespan}/2021-05.csv',
         f'tweet-sentiment/{timespan}/2021-06.csv']
datas = []
df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file, usecols = ['section','neg_count','neu_count','pos_count','open_price','trend'])
    datas.append(data)

# ファイルの結合
df = pd.concat(datas).reset_index(drop=True)
df = df.drop(columns=['section'])

# カラム名変更(区間No.割り当て)
df = df.rename(columns={"neg_count":"neg_count(n)", 
                        "neu_count":"neu_count(n)", 
                        "pos_count":"pos_count(n)", 
                        "open_price":"open_price(n)",
                        "trend":"trend(n)"})

# 目的変数
df['trend(n+1)'] = df['trend(n)'].shift(-1)

# 説明変数
df['open_price(n+1)'] = df['open_price(n)'].shift(-1)
if n >= 2:
    for i in range(1,n):
        df[f'neg_count(n-{i})'] = df['neg_count(n)'].shift(i)
        df[f'neu_count(n-{i})'] = df['neu_count(n)'].shift(i)
        df[f'pos_count(n-{i})'] = df['pos_count(n)'].shift(i)
        df[f'trend(n-{i})'] = df['trend(n)'].shift(i)
        if i==n-1: break
        df[f'open_price(n-{i})'] = df['open_price(n)'].shift(i)
else:
    df = df.drop(columns=['open_price(n)'])

        
print(len(df.columns.values))
print(df.columns.values)

11
['neg_count(n)' 'neu_count(n)' 'pos_count(n)' 'open_price(n)' 'trend(n)'
 'trend(n+1)' 'open_price(n+1)' 'neg_count(n-1)' 'neu_count(n-1)'
 'pos_count(n-1)' 'trend(n-1)']


In [39]:
print(df.dtypes)

neg_count(n)         int64
neu_count(n)         int64
pos_count(n)         int64
open_price(n)      float64
trend(n)             int64
trend(n+1)         float64
open_price(n+1)    float64
neg_count(n-1)     float64
neu_count(n-1)     float64
pos_count(n-1)     float64
trend(n-1)         float64
dtype: object
